In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings

warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random

pd.set_option('display.max_columns', None)

# 이름 전처리

In [256]:
df = pd.read_csv("AIS_data_preprocessed_drop_dupl_1125.csv")
df

,remained_distance,speed,time_voyage_data_received,destination,estimated_fuel_consumption,vessel_name,mmsi,navigation_info,timestamp,latitude,time_position_data_received,imo,heading,eta,CAPACITY,longitude
0,0.000000,18.2,36:18.0,MX LZC > KR PUS,11355.525720,RIODEJANEIRO EXPRESS,211111000,under way using engine,26:16.6,43.0744,18:59.0,9508902.0,294.0,9150500,0,-146.5312
1,0.000000,0.0,53:55.0,CNTAO,NaN,UMM SALAL,215237000,at anchor,26:16.6,35.7335,26:53.0,9525857.0,97.0,9071200,13296,121.0677
2,0.000000,0.0,11:00.0,CL IQQ,NaN,KUALA LUMPUR EXPRESS,218284000,moored,26:16.6,-20.2053,20:19.0,9343730.0,209.0,9040500,8606,-70.1563
3,0.000000,0.6,19:36.0,CAVAN,237157.744400,MSC LA SPEZIA,218400000,under way using engine,26:16.6,48.1475,23:44.0,9461403.0,253.0,9100500,14736,-126.0830
4,0.000000,18.4,16:55.0,CLVAP,63858.996790,NAVIGARE COLLECTOR,219059000,under way using engine,26:16.6,-34.8304,04:36.0,9779771.0,18.0,9060400,10100,-72.9862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127710,672.321619,12.7,2022-09-07 08:51:32,CNSHA,1747.198305,CSCL WINTER,477117400,under way using engine,2022-09-07 23:20:51.445000,29.6953,2022-09-07 14:16:08,9645877.0,230.0,9070900,10036,125.3178
127711,871.358312,0.3,2022-09-07 01:10:53,CNSHA,50506.400705,MSC LUCIANA,355283000,under way using engine,2022-09-07 23:20:51.445000,28.9129,2022-09-07 14:05:51,9398383.0,331.0,9080600,11668,122.9533
127712,4556.619324,0.1,2022-09-07 02:16:58,SGSIN AEW,357811.914697,AMALTHEA,255805781,at anchor,2022-09-07 23:20:51.445000,1.2542,2022-09-07 14:19:22,9397913.0,264.0,9061030,4178,103.8042
127713,5892.948969,0.3,2022-09-07 02:16:01,LKCMB ANCHORAGE,339184.436506,ZIM CHARLESTON,477714100,at anchor,2022-09-07 23:20:51.445000,6.9352,2022-09-07 14:17:35,9461506.0,281.0,8290230,8586,79.7244


In [257]:
df.loc[df.query('mmsi==667001506').index,
             "vessel_name"] = 'EVERGO 1'

In [258]:
import re

In [259]:
df['vessel_name'] = df['vessel_name'].apply(
    lambda x: re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", x))

In [260]:
df["vessel_name"] = df["vessel_name"].apply(
    lambda x: x.replace("'", ""))
df["vessel_name"] = df["vessel_name"].apply(
    lambda x: x.replace('"', ''))
df["vessel_name"] = df["vessel_name"].apply(
    lambda x: x.replace('\\', ''))

In [261]:
mmsi_list = list(dict.fromkeys(df['mmsi'].values))
cnt = 0
for mmsi in mmsi_list:
    if len(list(dict.fromkeys(df.query(f'mmsi=={mmsi}')["vessel_name"].values))) > 1:
#         print(list(dict.fromkeys(df_final.query(f'mmsi=={mmsi}')["vessel_name"].values)))
        cnt +=1

In [262]:
cnt

7

In [263]:
df.drop(df.query('vessel_name=="N4 B59 BIB 09 76"').index,
              inplace=True)

In [264]:
for mmsi in mmsi_list:
    if len(
            list(
                dict.fromkeys(
                    df.query(f'mmsi=={mmsi}')
                    ["vessel_name"].values))) > 1:
        real_vessel_name = df.query(
            f"mmsi=={mmsi}")["vessel_name"].value_counts().index[0]
        idx = df.query(f'mmsi=={mmsi}').index
        df.loc[idx, "vessel_name"] = real_vessel_name

In [265]:
mmsi_list = list(dict.fromkeys(df['mmsi'].values))
cnt = 0
for mmsi in mmsi_list:
    if len(list(dict.fromkeys(df.query(f'mmsi=={mmsi}')["vessel_name"].values))) > 1:
#         print(list(dict.fromkeys(df_final.query(f'mmsi=={mmsi}')["vessel_name"].values)))
        cnt +=1

In [266]:
cnt

0

In [267]:
whole_vessel_list = list(dict.fromkeys(df["vessel_name"].values))
len(whole_vessel_list)

209

In [268]:
df["vessel_name"].value_counts().tail(50)

MSC SILVANA           116
MSC DANIELA           115
MSC MONICA             99
NAVIGARE COLLECTOR     98
MSC HAILEY ANN III     90
TAYMA EXPRESS          89
MSC PEGASUS            70
MSC BETTINA            66
MSC CAPELLA            64
UMM SALAL              61
YM WELLBEING           52
MSC BEIJING            51
MSC VITA               45
OOCL BRAZIL            37
MSC DESIREE            37
PACIFIC TIANJIN        33
NAVIOS UTMOST          31
AL DHAIL               18
MSC TIPHAINE           17
MSC VILDA X            14
MSC AURIGA              8
MSC ANS                 7
MSC REBECCA             6
CONTI CHIVALRY          6
MSC BREMEN              5
MSC TORONTO             4
SHIJING                 4
MSC INDIA               4
LEO C                   3
XIN MEI ZHOU            3
MSC SHAY                3
MSC ROBERTA   V         3
COSCO SAO PAULO         3
MSC POLARIS             3
MSC TANIA               3
MSC DANIT               3
MSC EVEREST             3
MSC TARANTO             3
MSC BRANKA  

데이터가 100개 미만 삭제

In [269]:
df.query("vessel_name.str.contains('UTMOST')")

,remained_distance,speed,time_voyage_data_received,destination,estimated_fuel_consumption,vessel_name,mmsi,navigation_info,timestamp,latitude,time_position_data_received,imo,heading,eta,CAPACITY,longitude
90,0.000000,10.2,51:25.0,CNYTN,4826.311780,NAVIOS UTMOST,636012808,under way using engine,26:16.6,22.3279,52:39.0,9302621.0,332.0,9050700,8204,114.5227
307,0.000000,14.8,15:11.0,KRPUS,5329.171087,NAVIOS UTMOST,636012808,under way using engine,43:53.5,22.1973,29:31.0,9302621.0,107.0,9100600,8204,114.7718
3238,1513.705853,11.1,2022-09-07 03:46:51,KRPUS,3687.013838,NAVIOS UTMOST,636012808,under way using engine,2022-09-08 01:12:55.522076,24.2674,2022-09-07 16:12:28,9302621.0,60.0,9111100,8204,119.1112
3282,1503.993863,11.3,2022-09-07 04:34:55,KRPUS,3667.987702,NAVIOS UTMOST,636012808,under way using engine,2022-09-08 01:47:25.975678,24.3121,2022-09-07 16:44:02,9302621.0,60.0,9111100,8204,119.2069
3316,1494.092886,11.2,2022-09-07 05:16:59,KRPUS,3641.250895,NAVIOS UTMOST,636012808,under way using engine,2022-09-08 02:17:27.973888,24.3601,2022-09-07 17:15:30,9302621.0,58.0,9111100,8204,119.3010
3352,1484.257499,11.3,2022-09-07 05:16:59,KRPUS,3619.854034,NAVIOS UTMOST,636012808,under way using engine,2022-09-08 02:47:30.726782,24.4083,2022-09-07 17:46:41,9302621.0,60.0,9111100,8204,119.3939
3385,1477.015348,11.2,2022-09-07 05:16:59,KRPUS,3599.631262,NAVIOS UTMOST,636012808,under way using engine,2022-09-08 03:17:33.145027,24.4422,2022-09-07 18:10:00,9302621.0,60.0,9111100,8204,119.4649
3416,1466.834535,11.6,2022-09-07 06:35:09,KRPUS,3588.315738,NAVIOS UTMOST,636012808,under way using engine,2022-09-08 03:47:35.841497,24.4895,2022-09-07 18:42:30,9302621.0,58.0,9111100,8204,119.5655
3450,1456.480774,11.5,2022-09-07 07:11:13,KRPUS,3558.820792,NAVIOS UTMOST,636012808,under way using engine,2022-09-08 04:17:38.263142,24.5394,2022-09-07 19:14:42,9302621.0,53.0,9111100,8204,119.6653
3483,1445.775263,11.8,2022-09-07 07:35:17,KRPUS,3546.639170,NAVIOS UTMOST,636012808,under way using engine,2022-09-08 04:47:40.513111,24.5990,2022-09-07 19:45:49,9302621.0,53.0,9111100,8204,119.7562


In [40]:
drop_name_list = df["vessel_name"].value_counts()[df["vessel_name"].value_counts() < 50].index
drop_name_list

Index(['MSC VITA', 'OOCL BRAZIL', 'MSC DESIREE', 'PACIFIC TIANJIN',
       'NAVIOS UTMOST', 'AL DHAIL', 'MSC TIPHAINE', 'MSC VILDA X',
       'MSC AURIGA', 'MSC ANS', 'MSC REBECCA', 'CONTI CHIVALRY', 'MSC BREMEN',
       'MSC TORONTO', 'SHIJING', 'MSC INDIA', 'LEO C', 'XIN MEI ZHOU',
       'MSC SHAY', 'MSC ROBERTA   V', 'COSCO SAO PAULO', 'MSC POLARIS',
       'MSC TANIA', 'MSC DANIT', 'MSC EVEREST', 'MSC TARANTO', 'MSC BRANKA',
       'MSC MARGARITA', 'MSC NASSAU', 'COSCO NEW YORK', 'RIO GRANDE EXPRESS',
       'MSC SOLA', 'YM WELCOME', 'MSC TAMPA', 'CONTI ANNAPURNA', 'MSC ELLEN',
       'MUMBAI MAERSK', 'AL RIFFA'],
      dtype='object')

In [58]:
list(df.query(f"vessel_name == '{name}'").index)

[26,
 152,
 248,
 2053,
 2077,
 2111,
 2147,
 2175,
 3204,
 3206,
 3208,
 3209,
 3212,
 3215,
 3217,
 3218,
 3219,
 3221,
 3223,
 3225,
 3227,
 3236,
 3280,
 3314,
 3350,
 3383,
 3414,
 3448,
 3481,
 3630,
 3739,
 4353,
 127358,
 127388,
 127415,
 127437,
 127465,
 127498,
 127525,
 127547,
 127569,
 127599,
 127628,
 127656,
 127686]

In [67]:
drop_idx = []
for name in drop_name_list:
    for i in df.query(f"vessel_name == '{name}'").index:
        drop_idx.append(i)

In [70]:
drop_idx.sort()

In [73]:
df.drop(drop_idx, inplace=True)
df.reset_index(inplace=True, drop=True)

In [76]:
df

,remained_distance,speed,time_voyage_data_received,destination,estimated_fuel_consumption,vessel_name,mmsi,navigation_info,timestamp,latitude,time_position_data_received,imo,heading,eta,CAPACITY,longitude
0,0.000000,18.2,36:18.0,MX LZC > KR PUS,11355.525720,RIODEJANEIRO EXPRESS,211111000,under way using engine,26:16.6,43.0744,18:59.0,9508902.0,294.0,9150500,0,-146.5312
1,0.000000,0.0,53:55.0,CNTAO,NaN,UMM SALAL,215237000,at anchor,26:16.6,35.7335,26:53.0,9525857.0,97.0,9071200,13296,121.0677
2,0.000000,0.0,11:00.0,CL IQQ,NaN,KUALA LUMPUR EXPRESS,218284000,moored,26:16.6,-20.2053,20:19.0,9343730.0,209.0,9040500,8606,-70.1563
3,0.000000,0.6,19:36.0,CAVAN,237157.744400,MSC LA SPEZIA,218400000,under way using engine,26:16.6,48.1475,23:44.0,9461403.0,253.0,9100500,14736,-126.0830
4,0.000000,18.4,16:55.0,CLVAP,63858.996790,NAVIGARE COLLECTOR,219059000,under way using engine,26:16.6,-34.8304,04:36.0,9779771.0,18.0,9060400,10100,-72.9862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127381,672.321619,12.7,2022-09-07 08:51:32,CNSHA,1747.198305,CSCL WINTER,477117400,under way using engine,2022-09-07 23:20:51.445000,29.6953,2022-09-07 14:16:08,9645877.0,230.0,9070900,10036,125.3178
127382,871.358312,0.3,2022-09-07 01:10:53,CNSHA,50506.400705,MSC LUCIANA,355283000,under way using engine,2022-09-07 23:20:51.445000,28.9129,2022-09-07 14:05:51,9398383.0,331.0,9080600,11668,122.9533
127383,4556.619324,0.1,2022-09-07 02:16:58,SGSIN AEW,357811.914697,AMALTHEA,255805781,at anchor,2022-09-07 23:20:51.445000,1.2542,2022-09-07 14:19:22,9397913.0,264.0,9061030,4178,103.8042
127384,5892.948969,0.3,2022-09-07 02:16:01,LKCMB ANCHORAGE,339184.436506,ZIM CHARLESTON,477714100,at anchor,2022-09-07 23:20:51.445000,6.9352,2022-09-07 14:17:35,9461506.0,281.0,8290230,8586,79.7244


# destination 통일

In [77]:
df["locode_check"] = df["destination"].apply(lambda x: "yes"
                                             if len(str(x)) == 5 else "no")
df["destination"] = df["destination"].apply(lambda x: str(x).replace("'", ""))
df["destination"] = df["destination"].apply(lambda x: str(x).replace('"', ''))
df["destination"] = df["destination"].apply(lambda x: str(x).replace('\\', ''))
df["destination"] = df["destination"].apply(lambda x: x.upper())

In [78]:
df.loc[df.query(
    "destination.str.contains('BUS') | destination.str.contains('PUS') | destination.str.contains('KR') | destination.str.contains('BNP')"
).index, "destination"] = "KRBNP"

In [79]:
df["locode_check"] = df["destination"].apply(
    lambda x: "yes" if len(x.strip()) == 5 else "no")

In [80]:
dest_ok_idx = df[df["locode_check"] == "yes"].index
df_dest_ok = df.loc[dest_ok_idx, :]
df_dest_bad = df.drop(dest_ok_idx)

In [81]:
df_dest_bad["destination"]=df_dest_bad["destination"].apply(
    lambda x: re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", x))

In [82]:
dest_6_ok_idx = df_dest_bad[df_dest_bad["destination"].apply(
    lambda x: len(x) == 6)].query("destination.str.contains(' ')").index
df.loc[dest_6_ok_idx, "destination"] = df_dest_bad.loc[
    dest_6_ok_idx, "destination"].apply(lambda x: x.replace(" ", ""))

In [84]:
df["locode_check"] = df["destination"].apply(
    lambda x: "yes" if len(x.strip()) == 5 else "no")

In [86]:
dest_ok_idx = df[df["locode_check"] == "yes"].index
df_dest_ok = df.loc[dest_ok_idx, :]
df_dest_bad = df.drop(dest_ok_idx)

In [87]:
df_dest_bad["destination"] = df_dest_bad["destination"].apply(
    lambda x: x.replace("<", ">"))
df_dest_bad["destination"] = df_dest_bad["destination"].apply(
    lambda x: x.replace(">>", ">"))
df_dest_bad["destination"] = df_dest_bad["destination"].apply(
    lambda x: x.replace("->", ">"))
df_dest_bad["destination"] = df_dest_bad["destination"].apply(
    lambda x: x.replace("'", ""))
df_dest_bad["destination"] = df_dest_bad["destination"].apply(
    lambda x: x.replace('"', ""))

In [88]:
df_dest_bad["destination"] = df_dest_bad["destination"].apply(
    lambda x: x.split("-")[1].strip(" ") if "-" in x else x)
df_dest_bad["destination"] = df_dest_bad["destination"].apply(
    lambda x: x.split(">")[1].strip(" ") if ">" in x else x)

In [89]:
df_dest_bad.loc[df_dest_bad.query("destination.str.contains(' ')").index,
                "destination"] = df_dest_bad.loc[
                    df_dest_bad.query("destination.str.contains(' ')").index,
                    "destination"].apply(lambda x: x.split(" ")[1]
                                         if len(x.split(" ")[0]) == 5 and len(
                                             x.split(" ")[1]) == 5 else x)

In [90]:
df_dest_bad.drop(columns="locode_check", inplace=True)
df_dest_ok.drop(columns="locode_check", inplace=True)

In [91]:
df_pp = df_dest_ok.append(df_dest_bad)
df_pp

,remained_distance,speed,time_voyage_data_received,destination,estimated_fuel_consumption,vessel_name,mmsi,navigation_info,timestamp,latitude,time_position_data_received,imo,heading,eta,CAPACITY,longitude
0,0.000000,18.2,36:18.0,KRBNP,11355.525720,RIODEJANEIRO EXPRESS,211111000,under way using engine,26:16.6,43.0744,18:59.0,9508902.0,294.0,9150500,0,-146.5312
1,0.000000,0.0,53:55.0,CNTAO,NaN,UMM SALAL,215237000,at anchor,26:16.6,35.7335,26:53.0,9525857.0,97.0,9071200,13296,121.0677
2,0.000000,0.0,11:00.0,CLIQQ,NaN,KUALA LUMPUR EXPRESS,218284000,moored,26:16.6,-20.2053,20:19.0,9343730.0,209.0,9040500,8606,-70.1563
3,0.000000,0.6,19:36.0,CAVAN,237157.744400,MSC LA SPEZIA,218400000,under way using engine,26:16.6,48.1475,23:44.0,9461403.0,253.0,9100500,14736,-126.0830
4,0.000000,18.4,16:55.0,CLVAP,63858.996790,NAVIGARE COLLECTOR,219059000,under way using engine,26:16.6,-34.8304,04:36.0,9779771.0,18.0,9060400,10100,-72.9862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127373,649.993455,18.4,2022-09-07 02:06:39,TIANJINXINGANG,2454.927646,MSC CRISTINA,636016858,under way using engine,2022-09-07 23:20:51.445000,37.8963,2022-09-07 14:17:39,9465241.0,304.0,9080500,13000,122.5272
127377,2002.700993,0.1,2022-09-07 02:13:52,HK HKG,219888.235056,OOCL NEW YORK,477274700,moored,2022-09-07 23:20:51.445000,22.3360,2022-09-07 14:19:01,9198109.0,72.0,9071230,5770,114.1255
127380,5387.843710,13.5,2022-09-07 12:03:53,YOKOHAMA,13954.712098,HYUNDAI SATURN,232024771,under way using engine,2022-09-07 23:20:51.445000,44.4281,2022-09-07 13:57:51,9725146.0,272.0,9140100,10000,-167.7791
127383,4556.619324,0.1,2022-09-07 02:16:58,SGSIN AEW,357811.914697,AMALTHEA,255805781,at anchor,2022-09-07 23:20:51.445000,1.2542,2022-09-07 14:19:22,9397913.0,264.0,9061030,4178,103.8042


In [92]:
df_pp.reset_index(inplace=True, drop=True)

# eta

In [93]:
df_pp["eta"] = df_pp["eta"].apply(lambda x: "20220" + str(x)
                                  if len(str(x)) == 7 else str(x))

In [94]:
df_pp["eta"] = df_pp["eta"].apply(lambda x: "2022" + str(x)
                                  if len(str(x)) == 8 else str(x))

In [95]:
df_pp["eta_check"] = df_pp["eta"].apply(lambda x: "no"
                                        if len(x) is not 12 else "ok")

In [96]:
eta_error_drop_idx = df_pp[df_pp["eta_check"] == "no"].index
df_pp.drop(eta_error_drop_idx, inplace=True)

In [97]:
df_pp

,remained_distance,speed,time_voyage_data_received,destination,estimated_fuel_consumption,vessel_name,mmsi,navigation_info,timestamp,latitude,time_position_data_received,imo,heading,eta,CAPACITY,longitude,eta_check
0,0.000000,18.2,36:18.0,KRBNP,11355.525720,RIODEJANEIRO EXPRESS,211111000,under way using engine,26:16.6,43.0744,18:59.0,9508902.0,294.0,202209150500,0,-146.5312,ok
1,0.000000,0.0,53:55.0,CNTAO,NaN,UMM SALAL,215237000,at anchor,26:16.6,35.7335,26:53.0,9525857.0,97.0,202209071200,13296,121.0677,ok
2,0.000000,0.0,11:00.0,CLIQQ,NaN,KUALA LUMPUR EXPRESS,218284000,moored,26:16.6,-20.2053,20:19.0,9343730.0,209.0,202209040500,8606,-70.1563,ok
3,0.000000,0.6,19:36.0,CAVAN,237157.744400,MSC LA SPEZIA,218400000,under way using engine,26:16.6,48.1475,23:44.0,9461403.0,253.0,202209100500,14736,-126.0830,ok
4,0.000000,18.4,16:55.0,CLVAP,63858.996790,NAVIGARE COLLECTOR,219059000,under way using engine,26:16.6,-34.8304,04:36.0,9779771.0,18.0,202209060400,10100,-72.9862,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127381,649.993455,18.4,2022-09-07 02:06:39,TIANJINXINGANG,2454.927646,MSC CRISTINA,636016858,under way using engine,2022-09-07 23:20:51.445000,37.8963,2022-09-07 14:17:39,9465241.0,304.0,202209080500,13000,122.5272,ok
127382,2002.700993,0.1,2022-09-07 02:13:52,HK HKG,219888.235056,OOCL NEW YORK,477274700,moored,2022-09-07 23:20:51.445000,22.3360,2022-09-07 14:19:01,9198109.0,72.0,202209071230,5770,114.1255,ok
127383,5387.843710,13.5,2022-09-07 12:03:53,YOKOHAMA,13954.712098,HYUNDAI SATURN,232024771,under way using engine,2022-09-07 23:20:51.445000,44.4281,2022-09-07 13:57:51,9725146.0,272.0,202209140100,10000,-167.7791,ok
127384,4556.619324,0.1,2022-09-07 02:16:58,SGSIN AEW,357811.914697,AMALTHEA,255805781,at anchor,2022-09-07 23:20:51.445000,1.2542,2022-09-07 14:19:22,9397913.0,264.0,202209061030,4178,103.8042,ok


In [98]:
df_pp["eta_year"] =df_pp["eta"].apply(lambda x: int(x[:4]))
df_pp["eta_month"] =df_pp["eta"].apply(lambda x: int(x[4:6]))
df_pp["eta_day"] =df_pp["eta"].apply(lambda x: int(x[6:8]))
df_pp["eta_hour"] =df_pp["eta"].apply(lambda x: int(x[8:10]))
df_pp["eta_minute"] =df_pp["eta"].apply(lambda x: int(x[10:12]))


In [99]:
for idx in df_pp.query('eta_hour>24').index:
    eta_day = df_pp.loc[idx, "eta_day"]
    eta_hour = df_pp.loc[idx, "eta_hour"]
    eta_month = df_pp.loc[idx, "eta_month"]
    
    eta_hour -= 24
    eta_day += 1
    df_pp.loc[idx, "eta_day"] = eta_day
    df_pp.loc[idx, "eta_hour"] = eta_hour
    

In [100]:
df_pp["eta_month"] = df_pp["eta_month"].apply(lambda x: "0" + str(x)
                                              if len(str(x)) == 1 else str(x))
df_pp["eta_day"] = df_pp["eta_day"].apply(lambda x: "0" + str(x)
                                          if len(str(x)) == 1 else str(x))
df_pp["eta_hour"] = df_pp["eta_hour"].apply(lambda x: "0" + str(x)
                                            if len(str(x)) == 1 else str(x))
df_pp["eta_minute"] = df_pp["eta_minute"].apply(
    lambda x: "0" + str(x) if len(str(x)) == 1 else str(x))

In [101]:
df_pp["eta_year"] = df_pp["eta_year"].astype("str")

In [102]:
df_pp["eta"] = df_pp["eta_year"] + "-" + df_pp["eta_month"] + "-" + df_pp[
    "eta_day"] + " " + df_pp["eta_hour"] + ":" + df_pp["eta_minute"] + ":00"

In [103]:
df_pp["eta"] = pd.to_datetime(df_pp["eta"])

In [104]:
df_pp.drop([
    "eta_check", "eta_year", "eta_month", "eta_day", "eta_minute", "eta_hour"
],
           axis=1,
           inplace=True)

In [106]:
df_pp

,remained_distance,speed,time_voyage_data_received,destination,estimated_fuel_consumption,vessel_name,mmsi,navigation_info,timestamp,latitude,time_position_data_received,imo,heading,eta,CAPACITY,longitude
0,0.000000,18.2,36:18.0,KRBNP,11355.525720,RIODEJANEIRO EXPRESS,211111000,under way using engine,26:16.6,43.0744,18:59.0,9508902.0,294.0,2022-09-15 05:00:00,0,-146.5312
1,0.000000,0.0,53:55.0,CNTAO,NaN,UMM SALAL,215237000,at anchor,26:16.6,35.7335,26:53.0,9525857.0,97.0,2022-09-07 12:00:00,13296,121.0677
2,0.000000,0.0,11:00.0,CLIQQ,NaN,KUALA LUMPUR EXPRESS,218284000,moored,26:16.6,-20.2053,20:19.0,9343730.0,209.0,2022-09-04 05:00:00,8606,-70.1563
3,0.000000,0.6,19:36.0,CAVAN,237157.744400,MSC LA SPEZIA,218400000,under way using engine,26:16.6,48.1475,23:44.0,9461403.0,253.0,2022-09-10 05:00:00,14736,-126.0830
4,0.000000,18.4,16:55.0,CLVAP,63858.996790,NAVIGARE COLLECTOR,219059000,under way using engine,26:16.6,-34.8304,04:36.0,9779771.0,18.0,2022-09-06 04:00:00,10100,-72.9862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127381,649.993455,18.4,2022-09-07 02:06:39,TIANJINXINGANG,2454.927646,MSC CRISTINA,636016858,under way using engine,2022-09-07 23:20:51.445000,37.8963,2022-09-07 14:17:39,9465241.0,304.0,2022-09-08 05:00:00,13000,122.5272
127382,2002.700993,0.1,2022-09-07 02:13:52,HK HKG,219888.235056,OOCL NEW YORK,477274700,moored,2022-09-07 23:20:51.445000,22.3360,2022-09-07 14:19:01,9198109.0,72.0,2022-09-07 12:30:00,5770,114.1255
127383,5387.843710,13.5,2022-09-07 12:03:53,YOKOHAMA,13954.712098,HYUNDAI SATURN,232024771,under way using engine,2022-09-07 23:20:51.445000,44.4281,2022-09-07 13:57:51,9725146.0,272.0,2022-09-14 01:00:00,10000,-167.7791
127384,4556.619324,0.1,2022-09-07 02:16:58,SGSIN AEW,357811.914697,AMALTHEA,255805781,at anchor,2022-09-07 23:20:51.445000,1.2542,2022-09-07 14:19:22,9397913.0,264.0,2022-09-06 10:30:00,4178,103.8042


In [108]:
# for name in whole_vessel_list:
#     globals()[f"{name}_df_pp"] = df_pp.query(f"vessel_name == '{name}'")

In [109]:
# for i in globals():
#     if "_df_pp" in i:
#         print(i)

RIODEJANEIRO EXPRESS_df_pp
UMM SALAL_df_pp
KUALA LUMPUR EXPRESS_df_pp
MSC LA SPEZIA_df_pp
NAVIGARE COLLECTOR_df_pp
SOUTHAMPTON EXPRESS_df_pp
AL RIFFA_df_pp
VALOR_df_pp
VALUE_df_pp
VALENCE_df_pp
VANTAGE_df_pp
HYUNDAI SATURN_df_pp
MSC TAMPICO_df_pp
MSC PARIS_df_pp
AMALTHEA_df_pp
MSC ANTONELLA_df_pp
MSC CHANNE_df_pp
MSC DESIREE_df_pp
MSC BRANKA_df_pp
MSC AURIGA_df_pp
MSC ORION_df_pp
MSC VIRGO_df_pp
MSC CHARLESTON_df_pp
MSC BEIJING_df_pp
MSC TORONTO_df_pp
MSC BRUNELLA_df_pp
MSC VITA_df_pp
MSC SARA ELENA_df_pp
MERSIN EXPRESS_df_pp
MSC SANTA MARIA_df_pp
MSC ELA_df_pp
MSC LISA_df_pp
MSC CAMILLE_df_pp
MSC LUCIANA_df_pp
OOCL BRAZIL_df_pp
YM WELLBEING_df_pp
COSCO NEW YORK_df_pp
COSCO ASIA_df_pp
MSC ELENI_df_pp
MSC BERYL_df_pp
MSC SOLA_df_pp
MSC DAVAO III_df_pp
MSC SILVANA_df_pp
NAVIOS JASMINE_df_pp
MSC SHAY_df_pp
MSC CAPELLA_df_pp
MSC GIANNINA II_df_pp
XIN OU ZHOU_df_pp
XIN YA ZHOU_df_pp
XIN MEI ZHOU_df_pp
ZHUCHENGXINZHOU_df_pp
HENG HUI 6_df_pp
HMM BLESSING_df_pp
HEUNGA ULSAN_df_pp
SINOKOR NIIGA

In [110]:
df_pp["vessel_name"].value_counts()

ZHUCHENGXINZHOU      4056
COSCO FOS            3803
HEUNGA ULSAN         3730
PACIFIC SINGAPORE    3349
SINOKOR NIIGATA      3311
                     ... 
MSC BETTINA            66
MSC CAPELLA            64
UMM SALAL              61
YM WELLBEING           52
MSC BEIJING            51
Name: vessel_name, Length: 171, dtype: int64

# 기간 내 port-mis 데이터 확인

7/19~11/25까지

In [112]:
port_mis_raw_df = pd.read_excel("portmis.xlsx")
port_mis_raw_df

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
0,부산,VRAD2,OSG ALPHA,2015,8,최종,외항,출항,"7,167",2015-09-24 16:00,2022-08-10 00:00,2015-09-24 19:00,2015-10-06 11:27,기타,15ANJSM0229,WAM,08,M박지 M8 묘지,DALIAN,MAIZURU,풀컨테이너선,NaN,17,NaN,N,N,NaN,NaN
1,부산,UBWX9,TRANSIT SHAMORA,2022,10,최종,외항,출항,"7,170",2022-06-17 02:48,2022-08-12 13:39,2022-08-12 13:39,2022-08-16 15:28,기타,22SCHC0804E,WAN,03,남외항 N-3박지,VLADIVOSTOK,HAKATA/FUKUOKA,풀컨테이너선,NaN,15,NaN,Y,Y,NaN,NaN
2,부산,D5ZV9,HANSA HARBURG,2022,4,최종,외항,출항,"18,321",2022-06-23 16:47,2022-07-22 02:36,2022-07-22 02:36,2022-07-25 09:10,기타,22BLAK3009E,WAN,04,남외항 N-4박지,울산,KOBE,풀컨테이너선,NaN,21,NaN,N,N,NaN,NaN
3,부산,9HA2021,CMA CGM LIBRA,2022,3,최종,외항,출항,"131,332",2022-07-08 06:05,2022-07-26 06:56,2022-07-26 06:00,2022-07-26 10:27,기타,22CMAL0343E,MS6,03,신항 6부두 3선석,QINGDAO,QINGDAO,풀컨테이너선,NaN,27,NaN,Y,Y,NaN,NaN
4,부산,9V7987,MAERSK HAMBURG,2022,3,최종,외항,출항,"153,744",2022-07-15 17:20,2022-07-20 01:25,2022-07-20 01:00,2022-07-20 08:57,기타,22MAEU0629E,MSN,08,신항 2부두 5선석,NINGBO,DALIAN,풀컨테이너선,NaN,23,NaN,Y,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8694,부산,V7QI4,HYUNDAI DYNASTY,2022,21,변경,외항,입항,"52,581",2022-11-25 19:00,2022-11-26 21:00,2022-11-25 19:00,NaN,기타,22HDMUA525I,MBS,05,신선대부두 5선석,SINGAPORE,인천,풀컨테이너선,4.0,22,NaN,Y,Y,NaN,NaN
8695,부산,WCEU,PRESIDENT CLEVELAND,2022,13,최종,외항,입항,"75,015",2022-11-25 20:00,2022-11-27 04:00,2022-11-25 20:00,NaN,기타,22APLU1112I,MS5,01,신항 5부두 1선석,QINGDAO,"NAHA, OKINAWA",풀컨테이너선,NaN,23,NaN,Y,Y,NaN,NaN
8696,부산,DSRA2,SINOKOR NIIGATA,2022,44,변경,외항,입항,"6,490",2022-11-25 22:00,2022-11-26 21:00,2022-11-25 22:00,NaN,기타,22SNKO4907I,MBS,01,신선대부두 1선석,SHIBUSHI,IMARI,풀컨테이너선,9.0,10,NaN,N,N,NaN,NaN
8697,부산,3FIL,MAERSK ENSHI,2022,1,변경,외항,입항,"142,121",2022-11-25 23:00,2022-11-26 11:00,2022-11-25 23:00,NaN,기타,22MAEU0965I,MSN,08,신항 2부두 5선석,LONGBEACH,NINGBO,풀컨테이너선,NaN,24,NaN,Y,Y,NaN,NaN


In [114]:
port_mis_vsl_name = list(dict.fromkeys(port_mis_raw_df["선명"]))
port_mis_vsl_name

['OSG ALPHA',
 'TRANSIT SHAMORA',
 'HANSA HARBURG',
 'CMA  CGM LIBRA',
 'MAERSK HAMBURG',
 'DONGJIN HIGHNESS',
 'PACIFIC NINGBO',
 'SUNNY DAHLIA',
 'PANCON SUNSHINE',
 'DONGJIN FIDES',
 'CAPE ARTEMISIO',
 'PORT KLANG VOYAGER',
 'HEUNG-A XIAMEN',
 'KMTC BANGKOK',
 'LEVERKUSEN EXPRESS',
 'SINOKOR HONGKONG',
 'HYUNDAI COURAGE',
 'PANCON SUCCESS',
 'MSC ALINA',
 'KMTC JAKARTA',
 'UBENA',
 'HYUNDAI HOPE',
 'MAERSK KOLKATA',
 'INGENUITY',
 'EASLINE YANTAI',
 'COSCO SHIPPING SAKURA',
 'MSC KATIE',
 'PEGASUS PRIME',
 'STAR CLIPPER',
 'PACIFIC GENEVA',
 'ADDISON',
 'XIN LONG YUN 88',
 'ZHONG WAI YUN NINGBO',
 'SUNNY CANNA',
 'HENG YU',
 'CAPE FLINT',
 'HF WEALTH',
 'NAVIOS VERDE',
 'KMTC TOKYO',
 'CMA CGM BELLINI',
 'STARSHIP URSA',
 'NAGOYA TRADER',
 'HONG KONG EXPRESS',
 'SKY FLOWER',
 'MSC WASHINGTON',
 'CONCERTO',
 'STARSHIP TAURUS',
 'SUNNY LINDEN',
 'ONE COMPETENCE',
 'HEUNG-A HAIPHONG',
 'NIIGATA TRADER',
 'SAWASDEE SINGAPORE',
 'CMA CGM LOIRE',
 'PEGASUS PACER',
 'WIDE INDIA',
 'NYK DAN

In [115]:
len(port_mis_vsl_name)

1274

In [116]:
no_port_mis_name = []
for name in whole_vessel_list:
    if not name in port_mis_vsl_name:
        no_port_mis_name.append(name)

In [119]:
len(whole_vessel_list)

209

In [118]:
len(no_port_mis_name)

53

In [120]:
no_port_mis_name

['RIODEJANEIRO EXPRESS',
 'MSC CHANNE',
 'MSC BRANKA',
 'MSC CHARLESTON',
 'MSC TORONTO',
 'COSCO NEW YORK',
 'MSC SOLA',
 'NAVIOS JASMINE',
 'MSC SHAY',
 'ZHUCHENGXINZHOU',
 'HEUNGA ULSAN',
 'RIO GRANDE EXPRESS',
 'COSCO SAO PAULO',
 'PARANAGUA EXPRESS',
 'MSC POLARIS',
 'MSC TANIA',
 'MSC ANTONIA',
 'MSC ELLEN',
 'MSC MUNDRA VIII',
 'MSC EVEREST',
 'MSC NIMISHA III',
 'AKADIMOS',
 'MSC BETTINA',
 'MSC FREEPORT',
 'MSC MOON F',
 'MSC DEILA',
 'MSC MONICA',
 'MSC EMILY II',
 'MSC ALICE',
 'MSC MICHIGAN VII',
 'MSC MONTEREY',
 'MSC HEIDI',
 'MSC TRADER II',
 'MSC TIPHAINE',
 'MSC MEXICO V',
 'MSC BARBADOS',
 'MSC ELSA3',
 'MSC RAVENNA',
 'LONG BEACH TRADER',
 'MSC VILDA X',
 'MSC PETRA',
 'MSC REBECCA',
 'MSC FELIXSTOWE',
 'MSC DARWIN VI',
 'MSC ORNELLA',
 'MSC JASMINE',
 'MSC NERISSA',
 'MSC TRIESTE',
 'MSC ROBERTA   V',
 'MSC INDIA',
 'XIN NAN SHA',
 'MSC ANS',
 'MSC DANIT']

In [214]:
tsb_db = cx_Oracle.connect("tsb", "tsb00", "61.76.45.152:1521/BPADTDB")
klnet_db = cx_Oracle.connect("klnet", "klnet02", "61.76.45.152:1521/BPADTDB")
sju_db = cx_Oracle.connect("sju", "sju05", "61.76.45.152:1521/BPADTDB")


In [123]:
dt_vssl_f = pd.read_sql("select * from dt_vssl_f", klnet_db)
dt_vssl_f

,VSSL_KEY,CALL_LETTER,VSSL_NO,IMO_NO,MMSI_NO,VSSL_TP,VSSL_ENG_NM,VSSL_KOR_NM,FLAG,INT_TN,GRS_TN,NET_TN,DEAD_WEIGHT,VSSL_ALL_LEN,VSSL_WIDTH,VSSL_BUILD_DT,VSSL_DEF_BRD,VSSL_DEPTH,VSSL_LEN,UPDT_DATE
0,LADJ2,LADJ2,None,None,None,52,KONG GAS,KONG GAS,NO,9538.0,9538.0,3403.0,NaN,138.7,20.5,1975-10-01 00:00:00,9.1,11.90,128.50,NaT
1,/XUSD3,/XUSD3,None,None,None,39,DRACO,DRACO,KH,6941.0,6941.0,3601.0,NaN,120.0,19.8,1974-10-17 00:00:00,8.3,10.50,120.00,NaT
2,826000,826000,826000,None,None,79,KANG NAM B-1,강남비 1 호,KR,392.0,392.0,392.0,NaN,50.4,14.0,1982-12-01 00:00:00,3.1,3.50,50.40,NaT
3,826017,826017,826017,None,None,79,HYUNDAE NO.5,제5 현대호,KR,353.0,353.0,354.0,NaN,37.8,10.0,1982-02-01 00:00:00,3.2,3.20,37.80,NaT
4,835860,835860,835860,None,None,73,None,유광1호,KR,NaN,204.0,204.0,0.0,23.8,11.5,1983-03-01 00:00:00,3.2,4.27,23.88,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86624,222813,222813,222813,None,None,99,None,9한국호,KR,0.0,12.0,12.0,NaN,14.3,3.7,2005-07-29 00:00:00,0.0,1.11,14.29,2022-08-16 16:35:44
86625,9909613,BOSP6,None,9909613,None,55,HONGLI,HONGLI,CN,9916.0,9916.0,2974.0,NaN,119.6,19.8,2022-01-04 00:00:00,7.6,11.30,117.02,2022-11-16 15:48:12
86626,9917854,HZKL,None,9917854,None,78,HAYAT,HAYAT,SA,33817.0,33817.0,14334.0,NaN,193.8,35.0,2022-07-27 00:00:00,13.0,19.30,176.19,2022-10-04 18:09:23
86627,8961494,BI2587,None,8961494,None,92,HSIEH TA,HSIEH TA,TW,651.0,651.0,313.0,NaN,59.2,9.0,2000-11-11 00:00:00,3.7,4.00,59.20,2022-09-24 21:09:04


In [124]:
DT_VSSL_ARVL_F = pd.read_sql("select * from DT_VSSL_ARVL_F", klnet_db)
DT_VSSL_ARVL_F

,VSSL_KEY,YR,SER_NO,AGENT_CODE,AGENT_NM,OFFR_NM,RPT_DT,PERF_DT,BERTH_PLC_CODE_1,BERTH_PLC_CODE_2,NXT_PRT_1,NXT_PRT_2,TRANS_CRG_TN,DNGR_CRG_TN,GRS_TN,INT_TN,FLAG,VSSL_TP,VSSL_NM,SUB_CALL_LETTER_1,SUB_CALL_LETTER_2,SUB_TN_1,SUB_TN_2,ARVL_OBJ_CODE,ETD_DT,LAST_PRT_DEP_DT,LAST_PRT_1,LAST_PRT_2,ARVL_DT,UNLD_CRG_TN,TUG_YN,PLT_YN,CALL_LETTER,DEAD_WEIGHT,OILA1_CUR_LEFT_VOL,OILA1_IMO_TANK_VOL,OILA1_LD_YN,OILA1_TP_CODE,OILA2_CUR_LEFT_VOL,OILA2_IMO_TANK_VOL,OILA2_LD_YN,OILA2_TP_CODE,CRG_TN,UPDT_DATE
0,000209,2021,001,BSK3296,주식회사 에이치제이해운,김제은,2021-03-30 13:04:53,2021-03-30 13:08:01,OLT,01,KR,ZZZ,0.0,0.0,57.0,0.0,KR,61,103에이치제이,None,None,0.0,0.0,99,2021-01-31 00:00:00,2021-01-20 00:00:00,KR,ZZZ,2021-01-20 19:40:00,0.0,N,N,000209,0.00,NaN,NaN,None,None,NaN,NaN,None,None,0.0,NaT
1,000223,2021,001,YSZ0517,일광산업(주),고명관,2021-04-27 13:24:03,2021-04-29 15:54:00,MYF,11,KR,ZZZ,0.0,0.0,1612.0,1612.0,KR,31,해신,None,None,0.0,0.0,99,2021-04-27 13:00:00,2021-04-22 11:00:00,KR,ZZZ,2021-04-26 10:00:00,2170.0,Y,N,000223,0.00,NaN,NaN,None,None,NaN,NaN,None,None,2170.0,NaT
2,000223,2021,002,YSZ0517,일광산업(주),고명관,2021-05-26 09:29:28,2021-05-26 09:44:05,WAN,02,KR,ZZZ,0.0,0.0,1612.0,1612.0,KR,31,해신,None,None,0.0,0.0,08,2021-05-26 00:25:00,2021-05-25 15:30:00,KR,SCP,2021-05-25 22:57:00,0.0,Y,N,000223,0.00,NaN,NaN,None,None,NaN,NaN,None,None,0.0,NaT
3,000223,2021,003,YSZ0517,일광산업(주),고명관,2021-07-01 09:32:23,1899-12-31 00:00:00,WAN,02,KR,ZZZ,0.0,0.0,1612.0,1612.0,KR,31,해신,None,None,0.0,0.0,08,2021-07-01 13:00:00,2021-06-30 10:00:00,KR,ZZZ,2021-07-01 11:00:00,0.0,N,N,000223,0.00,NaN,NaN,None,None,NaN,NaN,None,None,0.0,NaT
4,000584,2021,001,BSK2939,진영선박,최길호,2021-01-18 10:22:49,2021-01-18 10:49:09,MB7,06,KR,ZZZ,0.0,0.0,39.0,39.0,KR,61,주안 2호,None,030197,522.0,0.0,99,2021-01-14 13:40:00,2021-01-14 02:15:00,KR,ZZZ,2021-01-14 07:15:00,0.0,N,N,000584,39.00,NaN,NaN,None,None,NaN,NaN,None,None,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23333,210031,2022,057,BSU2693,선영해운(주),강정재,2022-08-26 16:59:25,2022-08-29 10:13:25,MSN,02,KR,USN,0.0,2395.0,1653.0,0.0,KR,52,선영,None,None,0.0,0.0,02,2022-08-26 16:00:00,2022-08-23 09:20:00,KR,YOC,2022-08-23 18:10:00,2395.0,Y,N,210031,3333.48,NaN,NaN,None,None,NaN,NaN,None,None,2395.0,2022-08-29 16:47:23
23334,9793935,2022,003,KMC0360,하파그로이드코리아(주),CONSUL REYNANTE,2022-08-31 14:48:25,2022-09-01 08:42:05,MSN,02,CN,SHA,28166.0,1251.0,113828.0,113828.0,SG,41,NAVEGANTES EXPRESS,None,None,0.0,0.0,01,2022-09-02 14:00:00,2022-08-24 01:00:00,SG,SIN,2022-08-31 14:05:00,35833.0,Y,Y,9V6819,0.00,3228300.0,5231400.0,N,03,310800.0,1979600.0,N,04,73378.0,2022-09-05 08:50:34
23335,9261827,2022,002,KMC0326,엠에스씨코리아(주),WIN NAING,2022-09-12 03:52:36,2022-09-13 09:57:09,MSN,02,CN,TAO,2244.0,0.0,25703.0,25703.0,LR,41,MSC ANA CAMILA III,None,None,0.0,0.0,01,2022-09-13 07:00:00,2022-09-03 12:00:00,MY,TPP,2022-09-12 08:03:00,2372.0,Y,Y,5LBP2,0.00,1045200.0,1780600.0,N,03,202400.0,875900.0,N,04,10809.0,2022-09-14 16:29:47
23336,9842102,2022,004,KMC0326,엠에스씨코리아(주),Sorokotyaga Igor,2022-10-07 12:26:41,2022-10-11 08:47:15,MSN,03,MX,ZLO,19733.0,0.0,140976.0,140976.0,LR,41,MSC KANOKO,None,None,0.0,0.0,01,2022-10-09 15:00:00,2022-10-04 12:00:00,CN,NGB,2022-10-07 17:00:00,26267.0,Y,Y,D5TM3,0.00,4190000.0,NaN,None,03,429000.0,NaN,None,04,105003.0,2022-10-11 10:15:41


In [128]:
no_port_mis_name

['RIODEJANEIRO EXPRESS',
 'MSC CHANNE',
 'MSC BRANKA',
 'MSC CHARLESTON',
 'MSC TORONTO',
 'COSCO NEW YORK',
 'MSC SOLA',
 'NAVIOS JASMINE',
 'MSC SHAY',
 'ZHUCHENGXINZHOU',
 'HEUNGA ULSAN',
 'RIO GRANDE EXPRESS',
 'COSCO SAO PAULO',
 'PARANAGUA EXPRESS',
 'MSC POLARIS',
 'MSC TANIA',
 'MSC ANTONIA',
 'MSC ELLEN',
 'MSC MUNDRA VIII',
 'MSC EVEREST',
 'MSC NIMISHA III',
 'AKADIMOS',
 'MSC BETTINA',
 'MSC FREEPORT',
 'MSC MOON F',
 'MSC DEILA',
 'MSC MONICA',
 'MSC EMILY II',
 'MSC ALICE',
 'MSC MICHIGAN VII',
 'MSC MONTEREY',
 'MSC HEIDI',
 'MSC TRADER II',
 'MSC TIPHAINE',
 'MSC MEXICO V',
 'MSC BARBADOS',
 'MSC ELSA3',
 'MSC RAVENNA',
 'LONG BEACH TRADER',
 'MSC VILDA X',
 'MSC PETRA',
 'MSC REBECCA',
 'MSC FELIXSTOWE',
 'MSC DARWIN VI',
 'MSC ORNELLA',
 'MSC JASMINE',
 'MSC NERISSA',
 'MSC TRIESTE',
 'MSC ROBERTA   V',
 'MSC INDIA',
 'XIN NAN SHA',
 'MSC ANS',
 'MSC DANIT']

In [134]:
DT_VSSL_ARVL_F = DT_VSSL_ARVL_F[DT_VSSL_ARVL_F.ARVL_DT > datetime.datetime(year=2022, month=7, day=11)]

In [141]:
no_port_mis_name_2 = []
for name in no_port_mis_name : 
    d = DT_VSSL_ARVL_F.query(f"VSSL_NM == '{name}'")
    if len(d) == 0:
        no_port_mis_name_2.append(name)
    else:
        display(d)

,VSSL_KEY,YR,SER_NO,AGENT_CODE,AGENT_NM,OFFR_NM,RPT_DT,PERF_DT,BERTH_PLC_CODE_1,BERTH_PLC_CODE_2,NXT_PRT_1,NXT_PRT_2,TRANS_CRG_TN,DNGR_CRG_TN,GRS_TN,INT_TN,FLAG,VSSL_TP,VSSL_NM,SUB_CALL_LETTER_1,SUB_CALL_LETTER_2,SUB_TN_1,SUB_TN_2,ARVL_OBJ_CODE,ETD_DT,LAST_PRT_DEP_DT,LAST_PRT_1,LAST_PRT_2,ARVL_DT,UNLD_CRG_TN,TUG_YN,PLT_YN,CALL_LETTER,DEAD_WEIGHT,OILA1_CUR_LEFT_VOL,OILA1_IMO_TANK_VOL,OILA1_LD_YN,OILA1_TP_CODE,OILA2_CUR_LEFT_VOL,OILA2_IMO_TANK_VOL,OILA2_LD_YN,OILA2_TP_CODE,CRG_TN,UPDT_DATE
22853,9706308,2022,002,ICC2115,오션네트워크익스프레스코리아(주),NITA SORIN GEORGE,2022-07-12 10:55:15,2022-07-13 08:47:10,MSN,02,MX,ZLO,13329.0,1159.0,96424.0,96424.0,MT,41,AKADIMOS,None,None,0.0,0.0,01,2022-07-14 02:00:00,2022-07-07 23:30:00,CN,YAN,2022-07-12 06:50:00,20647.0,Y,Y,9HA5144,0.0,1658000.0,NaN,None,03,242000.0,NaN,None,04,65074.0,2022-07-14 16:22:10


,VSSL_KEY,YR,SER_NO,AGENT_CODE,AGENT_NM,OFFR_NM,RPT_DT,PERF_DT,BERTH_PLC_CODE_1,BERTH_PLC_CODE_2,NXT_PRT_1,NXT_PRT_2,TRANS_CRG_TN,DNGR_CRG_TN,GRS_TN,INT_TN,FLAG,VSSL_TP,VSSL_NM,SUB_CALL_LETTER_1,SUB_CALL_LETTER_2,SUB_TN_1,SUB_TN_2,ARVL_OBJ_CODE,ETD_DT,LAST_PRT_DEP_DT,LAST_PRT_1,LAST_PRT_2,ARVL_DT,UNLD_CRG_TN,TUG_YN,PLT_YN,CALL_LETTER,DEAD_WEIGHT,OILA1_CUR_LEFT_VOL,OILA1_IMO_TANK_VOL,OILA1_LD_YN,OILA1_TP_CODE,OILA2_CUR_LEFT_VOL,OILA2_IMO_TANK_VOL,OILA2_LD_YN,OILA2_TP_CODE,CRG_TN,UPDT_DATE
23227,9236444,2022,001,KMC0326,엠에스씨코리아(주),MIRAMONTES ASTERIO,2022-10-12 09:29:03,NaT,MSN,01,CN,SHA,NaN,NaN,16802.0,16802.0,LR,41,MSC EMILY II,None,None,0.0,0.0,01,2022-10-15 22:00:00,2022-10-13 12:00:00,CN,TAO,2022-10-15 04:00:00,5000.0,Y,Y,5LDV5,NaN,NaN,NaN,None,None,NaN,NaN,None,None,10000.0,2022-10-12 10:03:41


,VSSL_KEY,YR,SER_NO,AGENT_CODE,AGENT_NM,OFFR_NM,RPT_DT,PERF_DT,BERTH_PLC_CODE_1,BERTH_PLC_CODE_2,NXT_PRT_1,NXT_PRT_2,TRANS_CRG_TN,DNGR_CRG_TN,GRS_TN,INT_TN,FLAG,VSSL_TP,VSSL_NM,SUB_CALL_LETTER_1,SUB_CALL_LETTER_2,SUB_TN_1,SUB_TN_2,ARVL_OBJ_CODE,ETD_DT,LAST_PRT_DEP_DT,LAST_PRT_1,LAST_PRT_2,ARVL_DT,UNLD_CRG_TN,TUG_YN,PLT_YN,CALL_LETTER,DEAD_WEIGHT,OILA1_CUR_LEFT_VOL,OILA1_IMO_TANK_VOL,OILA1_LD_YN,OILA1_TP_CODE,OILA2_CUR_LEFT_VOL,OILA2_IMO_TANK_VOL,OILA2_LD_YN,OILA2_TP_CODE,CRG_TN,UPDT_DATE
23125,9484431,2022,003,KMC0326,엠에스씨코리아(주),Lukanowski Artur Emanuel,2022-11-25 08:43:34,NaT,MSN,03,CN,NGB,NaN,NaN,153115.0,153115.0,LR,41,MSC RAVENNA,None,None,0.0,0.0,01,2022-11-29 02:00:00,2022-11-25 12:00:00,CN,TXG,2022-11-28 02:00:00,10000.0,Y,Y,A8ZU8,NaN,NaN,NaN,None,None,NaN,NaN,None,None,20000.0,2022-11-25 09:04:51


,VSSL_KEY,YR,SER_NO,AGENT_CODE,AGENT_NM,OFFR_NM,RPT_DT,PERF_DT,BERTH_PLC_CODE_1,BERTH_PLC_CODE_2,NXT_PRT_1,NXT_PRT_2,TRANS_CRG_TN,DNGR_CRG_TN,GRS_TN,INT_TN,FLAG,VSSL_TP,VSSL_NM,SUB_CALL_LETTER_1,SUB_CALL_LETTER_2,SUB_TN_1,SUB_TN_2,ARVL_OBJ_CODE,ETD_DT,LAST_PRT_DEP_DT,LAST_PRT_1,LAST_PRT_2,ARVL_DT,UNLD_CRG_TN,TUG_YN,PLT_YN,CALL_LETTER,DEAD_WEIGHT,OILA1_CUR_LEFT_VOL,OILA1_IMO_TANK_VOL,OILA1_LD_YN,OILA1_TP_CODE,OILA2_CUR_LEFT_VOL,OILA2_IMO_TANK_VOL,OILA2_LD_YN,OILA2_TP_CODE,CRG_TN,UPDT_DATE
23114,8420907,2022,001,KMC0326,엠에스씨코리아(주),Chernysh Mykola,2022-11-23 10:02:21,NaT,MSN,02,CN,TAO,NaN,NaN,31430.0,31430.0,PA,41,MSC JASMINE,None,None,0.0,0.0,01,2022-11-27 00:00:00,2022-11-18 12:00:00,SG,SIN,2022-11-26,10000.0,Y,Y,H3OB,NaN,NaN,NaN,None,None,NaN,NaN,None,None,20000.0,2022-11-23 10:10:54


In [171]:
tos_vessel = pd.read_sql("select * from tos_vessel", tsb_db)
tos_vessel

,TML_NUM,TML_CD,TML_CUD,VSL_CD,VSL_NM,LOA,WIDTH,DECK_CAPACITY,HOLD_CAPACITY,CAPACITY,CALL_SIGN
0,2665,PNIT,U,YMMT,YM MILESTONE,30560,4000,11,11,6589,A8ZC9
1,2666,PNIT,U,OZRE,SEASPAN BREEZE,33700,4820,11,11,10100,VRNL8
2,2791,PNIT,U,MGIR,MAERSK GIRONDE,29208,3220,11,11,4318,ZQAY4
3,2792,PNIT,U,MKIM,MAERSK KIMI,29990,4283,13,9,7060,PDHO
4,2793,PNIT,U,HNFL,HANJIN FLORIDA,22800,3730,9,11,3674,D5CD3
...,...,...,...,...,...,...,...,...,...,...,...
2947,5809,PNIT,U,ORI1,RITA,20730,3224,9,11,2785,CQIS4
2948,5810,PNIT,U,MBRS,BOMAR SPRING,21100,3200,11,11,2732,A8JY8
2949,5811,PNIT,U,MPLS,MSC POLARIS,29995,3710,11,11,4809,D5EF8
2950,5812,PNIT,U,UNID,EVER UNITED,28500,4000,11,11,4800,9V7957


In [169]:
tos_plan_berth = pd.read_sql("select * from tos_plan_berth", tsb_db)

In [170]:
tos_plan_berth

,TML_NUM,TML_CD,TML_CUD,VSL_CD,CALL_YEAR,CALL_SEQ,VOYAGE,PTNR_CODE,BERTH_NO,ALONG_SIDE,FROM_BITT,TO_BITT,ETA,ETB,ETW,ETC,ETD,ATA,ATB,ATW,ATC,ATD,YARD_OPEN,YARD_CLOSE,DIS_QTY,LOAD_QTY,UPDATE_TIME
0,807,PNIT,C,VLIX,2021,002,VLIX002,HLC,T2,P,20,38,2021-11-07 23:00:00,2021-11-08 07:00:00,NaT,None,2021-11-09 18:00:00,None,2021-11-08 07:20:00,2021-11-08 08:31:00,2021-11-09 16:09:00,2021-11-09 18:00:00,2021-11-03 00:00:00,2021-11-09 00:00:00,1580,2027,2021-11-09 17:23:31
1,808,PNIT,C,MIDA,2021,014,MIDA014,MSC,T2,P,23,36,2021-11-17 20:00:00,2021-11-17 22:00:00,NaT,None,2021-11-18 16:00:00,None,2021-11-17 22:10:00,2021-11-17 23:13:00,2021-11-18 15:46:00,2021-11-18 16:00:00,2021-11-12 22:00:00,2021-11-17 11:00:00,136,759,2021-11-18 16:22:47
2,809,PNIT,C,MTRI,2021,001,MTRI001,MSC,T2,P,19,42,2021-10-04 06:00:00,2021-10-04 07:00:00,NaT,None,2021-10-05 13:00:00,None,2021-10-04 07:20:00,2021-10-04 09:03:00,2021-10-05 12:00:00,2021-10-05 13:00:00,2021-10-01 07:00:00,2021-10-03 01:00:00,2058,2200,2021-11-03 15:27:50
3,810,PNIT,U,XIOZ,2021,003,XIOZ003,COS,T1,P,00,20,2021-12-11 19:00:00,2021-12-12 02:00:00,NaT,None,2021-12-13 00:00:00,None,2021-12-12 02:00:00,2021-12-12 03:17:00,2021-12-12 23:23:00,2021-12-13 00:00:00,2021-12-08 20:00:00,2021-12-11 18:00:00,668,967,2021-12-22 11:48:58
4,811,PNIT,U,ZBD5,2021,002,ZBD5002,ZIM,T1,P,03,18,2021-12-29 23:00:00,2021-12-30 01:00:00,NaT,None,2021-12-30 17:00:00,None,2021-12-30 01:10:00,2021-12-30 02:56:00,2021-12-30 16:31:00,2021-12-30 17:00:00,2021-12-27 01:00:00,2021-12-29 08:00:00,627,356,2021-12-31 10:16:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,2493,PNIT,D,MNDA,2022,002,MNDA002,MSC,T3,P,None,None,2022-12-03 21:00:00,2022-12-03 22:00:00,NaT,None,2022-12-04 16:00:00,None,NaT,NaT,NaT,NaT,2022-11-03 22:00:00,2022-12-03 12:00:00,1200,1300,NaT
1461,2494,PNIT,U,MRII,2022,002,MRII002,MSC,T3,P,None,None,2022-12-09 11:00:00,2022-12-09 12:00:00,NaT,None,2022-12-10 05:00:00,None,NaT,NaT,NaT,NaT,2022-12-02 12:00:00,2022-12-09 02:00:00,500,800,2022-11-25 14:06:28
1462,2495,PNIT,U,MDVO,2022,013,MDVO013,MSC,T3,P,43,57,2022-12-13 02:00:00,2022-12-13 03:00:00,NaT,None,2022-12-13 20:00:00,None,NaT,NaT,NaT,NaT,2022-12-06 03:00:00,2022-12-12 17:00:00,500,0,2022-11-25 14:06:29
1463,2496,PNIT,U,MACM,2022,004,MACM004,MSC,T3,P,43,57,2022-12-17 11:00:00,2022-12-17 12:00:00,NaT,None,2022-12-18 05:00:00,None,NaT,NaT,NaT,NaT,2022-12-10 12:00:00,2022-12-17 02:00:00,0,800,2022-11-25 14:06:30


In [148]:
DT_VSSL_ARVL_F = DT_VSSL_ARVL_F.drop(columns=["AGENT_CODE","AGENT_NM",'OFFR_NM','OILA1_CUR_LEFT_VOL','OILA1_IMO_TANK_VOL','OILA1_LD_YN','OILA1_TP_CODE','OILA2_CUR_LEFT_VOL','OILA2_IMO_TANK_VOL','OILA2_LD_YN','OILA2_TP_CODE','UNLD_CRG_TN','TUG_YN','PLT_YN'])

In [154]:
DT_VSSL_ARVL_F

,VSSL_KEY,YR,SER_NO,RPT_DT,PERF_DT,BERTH_PLC_CODE_1,BERTH_PLC_CODE_2,NXT_PRT_1,NXT_PRT_2,TRANS_CRG_TN,DNGR_CRG_TN,GRS_TN,INT_TN,FLAG,VSSL_TP,VSSL_NM,SUB_CALL_LETTER_1,SUB_CALL_LETTER_2,SUB_TN_1,SUB_TN_2,ARVL_OBJ_CODE,ETD_DT,LAST_PRT_DEP_DT,LAST_PRT_1,LAST_PRT_2,ARVL_DT,CALL_LETTER,DEAD_WEIGHT,CRG_TN,UPDT_DATE
22724,BPKD,2022,003,2022-07-17 16:21:45,2022-07-18 09:37:58,MSN,01,MX,ESE,2943.0,17.0,90757.0,90757.0,CN,41,XIN FEI ZHOU,None,None,0.0,0.0,01,2022-07-18 00:01:00,2022-07-14 01:00:00,CN,TAO,2022-07-17 04:00:00,BPKD,0.00,72410.0,2022-07-18 11:22:06
22725,9628180,2022,006,2022-07-21 09:07:53,2022-07-22 13:10:08,MSN,02,JP,YOK,19319.0,1488.0,95390.0,95390.0,MT,41,VALENCE,None,None,0.0,0.0,01,2022-07-22 15:00:00,2022-07-15 01:00:00,CN,TAO,2022-07-20 23:55:00,9HA3398,0.00,61627.0,2022-07-25 14:46:27
22727,9289051,2022,006,2022-08-20 08:29:56,2022-08-22 08:21:08,MSN,03,RU,VVO,11271.0,0.0,18334.0,18334.0,PA,41,MSC GIANNINA II,None,None,0.0,0.0,01,2022-08-21 09:00:00,2022-08-17 12:00:00,RU,VVO,2022-08-20 12:40:00,3FIP4,0.00,12644.0,2022-08-22 09:47:09
22728,210024,2022,079,2022-08-29 10:50:14,NaT,MW5,01,KR,USN,0.0,1250.0,756.0,0.0,KR,52,송림호,None,None,0.0,0.0,02,2022-08-27 06:10:00,2022-08-26 08:20:00,KR,USN,2022-08-26 12:20:00,210024,1901.00,1250.0,2022-08-29 10:59:18
22729,9857157,2022,001,2022-09-27 12:57:36,2022-09-27 17:32:57,MSN,03,MX,ZLO,15127.0,0.0,149525.0,149525.0,PT,41,MSC ORION,None,None,0.0,0.0,01,2022-09-29 01:00:00,2022-09-24 12:00:00,CN,NGB,2022-09-27 16:35:00,CQEB8,0.00,98988.0,2022-09-29 10:57:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23333,210031,2022,057,2022-08-26 16:59:25,2022-08-29 10:13:25,MSN,02,KR,USN,0.0,2395.0,1653.0,0.0,KR,52,선영,None,None,0.0,0.0,02,2022-08-26 16:00:00,2022-08-23 09:20:00,KR,YOC,2022-08-23 18:10:00,210031,3333.48,2395.0,2022-08-29 16:47:23
23334,9793935,2022,003,2022-08-31 14:48:25,2022-09-01 08:42:05,MSN,02,CN,SHA,28166.0,1251.0,113828.0,113828.0,SG,41,NAVEGANTES EXPRESS,None,None,0.0,0.0,01,2022-09-02 14:00:00,2022-08-24 01:00:00,SG,SIN,2022-08-31 14:05:00,9V6819,0.00,73378.0,2022-09-05 08:50:34
23335,9261827,2022,002,2022-09-12 03:52:36,2022-09-13 09:57:09,MSN,02,CN,TAO,2244.0,0.0,25703.0,25703.0,LR,41,MSC ANA CAMILA III,None,None,0.0,0.0,01,2022-09-13 07:00:00,2022-09-03 12:00:00,MY,TPP,2022-09-12 08:03:00,5LBP2,0.00,10809.0,2022-09-14 16:29:47
23336,9842102,2022,004,2022-10-07 12:26:41,2022-10-11 08:47:15,MSN,03,MX,ZLO,19733.0,0.0,140976.0,140976.0,LR,41,MSC KANOKO,None,None,0.0,0.0,01,2022-10-09 15:00:00,2022-10-04 12:00:00,CN,NGB,2022-10-07 17:00:00,D5TM3,0.00,105003.0,2022-10-11 10:15:41


In [155]:
DT_VSSL_ARVL_F[["VSSL_KEY",'CALL_LETTER',"VSSL_NM"]]

,VSSL_KEY,CALL_LETTER,VSSL_NM
22724,BPKD,BPKD,XIN FEI ZHOU
22725,9628180,9HA3398,VALENCE
22727,9289051,3FIP4,MSC GIANNINA II
22728,210024,210024,송림호
22729,9857157,CQEB8,MSC ORION
...,...,...,...
23333,210031,210031,선영
23334,9793935,9V6819,NAVEGANTES EXPRESS
23335,9261827,5LBP2,MSC ANA CAMILA III
23336,9842102,D5TM3,MSC KANOKO


In [142]:
no_port_mis_name_2

['RIODEJANEIRO EXPRESS',
 'MSC CHANNE',
 'MSC BRANKA',
 'MSC CHARLESTON',
 'MSC TORONTO',
 'COSCO NEW YORK',
 'MSC SOLA',
 'NAVIOS JASMINE',
 'MSC SHAY',
 'ZHUCHENGXINZHOU',
 'HEUNGA ULSAN',
 'RIO GRANDE EXPRESS',
 'COSCO SAO PAULO',
 'PARANAGUA EXPRESS',
 'MSC POLARIS',
 'MSC TANIA',
 'MSC ANTONIA',
 'MSC ELLEN',
 'MSC MUNDRA VIII',
 'MSC EVEREST',
 'MSC NIMISHA III',
 'MSC BETTINA',
 'MSC FREEPORT',
 'MSC MOON F',
 'MSC DEILA',
 'MSC MONICA',
 'MSC ALICE',
 'MSC MICHIGAN VII',
 'MSC MONTEREY',
 'MSC HEIDI',
 'MSC TRADER II',
 'MSC TIPHAINE',
 'MSC MEXICO V',
 'MSC BARBADOS',
 'MSC ELSA3',
 'LONG BEACH TRADER',
 'MSC VILDA X',
 'MSC PETRA',
 'MSC REBECCA',
 'MSC FELIXSTOWE',
 'MSC DARWIN VI',
 'MSC ORNELLA',
 'MSC NERISSA',
 'MSC TRIESTE',
 'MSC ROBERTA   V',
 'MSC INDIA',
 'XIN NAN SHA',
 'MSC ANS',
 'MSC DANIT']

In [162]:
DT_VSSL_ARVL_F.query(f'VSSL_NM == "RIO DE JANEIRO EXPRESS"')

,VSSL_KEY,YR,SER_NO,RPT_DT,PERF_DT,BERTH_PLC_CODE_1,BERTH_PLC_CODE_2,NXT_PRT_1,NXT_PRT_2,TRANS_CRG_TN,DNGR_CRG_TN,GRS_TN,INT_TN,FLAG,VSSL_TP,VSSL_NM,SUB_CALL_LETTER_1,SUB_CALL_LETTER_2,SUB_TN_1,SUB_TN_2,ARVL_OBJ_CODE,ETD_DT,LAST_PRT_DEP_DT,LAST_PRT_1,LAST_PRT_2,ARVL_DT,CALL_LETTER,DEAD_WEIGHT,CRG_TN,UPDT_DATE
22904,9508902,2022,001,2022-08-09 09:35:43,2022-08-09 09:39:09,MSN,03,MX,ZLO,44.0,0.0,123560.0,123560.0,DE,41,RIO DE JANEIRO EXPRESS,None,None,0.0,0.0,01,2022-08-09 18:00:00,2022-08-05 01:00:00,CN,NGB,2022-08-09,DGXS2,0.0,28064.0,2022-08-10 12:34:47


# pnit 접안 히스토리

In [157]:
pnit_berth_history_df = pd.read_excel("pnit_berth_history.xlsx")
pnit_berth_history_df

,선석,선사,모선항차,선사항차,Head (Bridge) Stern,선명,ROUTE,반입마감시한,접안(예정)일시,출항(예정)일시,양하,적하,Shift,상태
0,T3(P),HMM,HHST006,0033E/0033E,41 (49) 61,HYUNDAI SATURN,AN3E,2022-07-08 22:00:00,2022-07-09 10:40:00,2022-07-11 12:00:00,1805,2999,4,DEPARTED
1,T2(P),HLC,CAUX003,2215W/2226E,23 (31) 42,CAUTIN,AN1,2022-07-09 13:00:00,2022-07-10 00:40:00,2022-07-12 05:00:00,1279,2134,896,DEPARTED
2,T1(S),MSC,MGHE003,UK221R/FI227A,21 (08) 03,MSC ALGHERO,AS2,2022-07-10 02:00:00,2022-07-10 19:25:00,2022-07-12 18:00:00,1615,1510,12,DEPARTED
3,T3(P),HLC,TAYX003,022W/022W,41 (50) 63,TAYMA EXPRESS,AG3,2022-07-11 04:00:00,2022-07-11 14:15:00,2022-07-13 11:00:00,134,2731,52,DEPARTED
4,T2(P),ONE,OAKD002,2227E/2227E,23 (36) 42,AKADIMOS,AN1,2022-07-11 19:00:00,2022-07-12 07:00:00,2022-07-13 20:00:00,1253,2032,344,DEPARTED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,T3(P),MSC,MPER007,FA240R/FA240R,41 (50) 63,MSC PERLE,ASPA2W,2022-11-28 11:00:00,2022-11-28 21:00:00,2022-11-30 02:00:00,2000,1000,0,PLANNED
279,T1(P),COS,HEHU003,009E/009E,02 (15) 20,HENG HUI 6,WSA3,2022-11-29 00:00:00,2022-11-29 10:00:00,2022-11-30 10:00:00,195,1000,0,PLANNED
280,T2(P),COH,CFOS027,2248W/2249E,24 (35) 40,COSCO FOS,AK3,2022-11-29 02:00:00,2022-11-29 12:00:00,2022-11-30 12:00:00,1700,900,0,PLANNED
281,T3(P),HLC,MVEX004,2234E/2247W,42 (49) 62,NAVEGANTES EXPRESS,AS2,2022-11-29 20:00:00,2022-11-30 06:00:00,2022-12-02 06:00:00,2596,1600,24,PLANNED


In [166]:
pnit_berth_history_df["VSL_CD"] = pnit_berth_history_df["모선항차"].apply(lambda x : x[:4])

In [167]:
pnit_berth_history_df

,선석,선사,모선항차,선사항차,Head (Bridge) Stern,선명,ROUTE,반입마감시한,접안(예정)일시,출항(예정)일시,양하,적하,Shift,상태,VSL_CD
0,T3(P),HMM,HHST006,0033E/0033E,41 (49) 61,HYUNDAI SATURN,AN3E,2022-07-08 22:00:00,2022-07-09 10:40:00,2022-07-11 12:00:00,1805,2999,4,DEPARTED,HHST
1,T2(P),HLC,CAUX003,2215W/2226E,23 (31) 42,CAUTIN,AN1,2022-07-09 13:00:00,2022-07-10 00:40:00,2022-07-12 05:00:00,1279,2134,896,DEPARTED,CAUX
2,T1(S),MSC,MGHE003,UK221R/FI227A,21 (08) 03,MSC ALGHERO,AS2,2022-07-10 02:00:00,2022-07-10 19:25:00,2022-07-12 18:00:00,1615,1510,12,DEPARTED,MGHE
3,T3(P),HLC,TAYX003,022W/022W,41 (50) 63,TAYMA EXPRESS,AG3,2022-07-11 04:00:00,2022-07-11 14:15:00,2022-07-13 11:00:00,134,2731,52,DEPARTED,TAYX
4,T2(P),ONE,OAKD002,2227E/2227E,23 (36) 42,AKADIMOS,AN1,2022-07-11 19:00:00,2022-07-12 07:00:00,2022-07-13 20:00:00,1253,2032,344,DEPARTED,OAKD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,T3(P),MSC,MPER007,FA240R/FA240R,41 (50) 63,MSC PERLE,ASPA2W,2022-11-28 11:00:00,2022-11-28 21:00:00,2022-11-30 02:00:00,2000,1000,0,PLANNED,MPER
279,T1(P),COS,HEHU003,009E/009E,02 (15) 20,HENG HUI 6,WSA3,2022-11-29 00:00:00,2022-11-29 10:00:00,2022-11-30 10:00:00,195,1000,0,PLANNED,HEHU
280,T2(P),COH,CFOS027,2248W/2249E,24 (35) 40,COSCO FOS,AK3,2022-11-29 02:00:00,2022-11-29 12:00:00,2022-11-30 12:00:00,1700,900,0,PLANNED,CFOS
281,T3(P),HLC,MVEX004,2234E/2247W,42 (49) 62,NAVEGANTES EXPRESS,AS2,2022-11-29 20:00:00,2022-11-30 06:00:00,2022-12-02 06:00:00,2596,1600,24,PLANNED,MVEX


In [188]:
pnit_berth_history_df.query("선명.str.contains('EXPRESS')")

,선석,선사,모선항차,선사항차,Head (Bridge) Stern,선명,ROUTE,반입마감시한,접안(예정)일시,출항(예정)일시,양하,적하,Shift,상태,VSL_CD
3,T3(P),HLC,TAYX003,022W/022W,41 (50) 63,TAYMA EXPRESS,AG3,2022-07-11 04:00:00,2022-07-11 14:15:00,2022-07-13 11:00:00,134,2731,52,DEPARTED,TAYX
36,T3(P),HLC,SPXX003,030E/031W,41 (50) 63,SOUTHAMPTON EXPRESS,AG3,2022-07-28 09:00:00,2022-07-28 09:05:00,2022-07-30 08:00:00,1554,2368,66,DEPARTED,SPXX
54,T2(P),HLC,SDXX002,2218E/2231W,22 (29) 42,SALVADOR EXPRESS,AS2,2022-08-04 22:00:00,2022-08-05 05:20:00,2022-08-06 20:00:00,1829,1071,604,DEPARTED,SDXX
55,T1(P),HLC,KUXX003,2220W/2231E,02 (17) 23,KUALA LUMPUR EXPRESS,AN1,2022-08-05 16:00:00,2022-08-06 02:15:00,2022-08-07 15:00:00,1334,1465,114,DEPARTED,KUXX
60,T3(P),HLC,RIXX001,001E/001E,41 (49) 62,RIO DE JANEIRO EXPRESS,SPT,2022-08-08 08:00:00,2022-08-09 00:25:00,2022-08-09 21:00:00,2,1134,2,DEPARTED,RIXX
102,T2(P),HLC,MVEX003,2221E/2234W,22 (29) 42,NAVEGANTES EXPRESS,AS2,2022-08-31 01:00:00,2022-08-31 14:20:00,2022-09-02 22:00:00,1990,1558,972,DEPARTED,MVEX
127,T3(P),HLC,TAYX004,022E/023W,41 (50) 63,TAYMA EXPRESS,AG3,2022-09-15 02:00:00,2022-09-15 12:10:00,2022-09-16 22:00:00,1795,881,176,DEPARTED,TAYX
128,T1(P),HLC,MRSX001,001E/001E,13 (23) 25,MERSIN EXPRESS,SPT,2022-09-15 03:00:00,2022-09-15 16:25:00,2022-09-16 04:00:00,108,266,58,DEPARTED,MRSX
155,T2(P),HLC,SPXX004,031E/032W,20 (29) 42,SOUTHAMPTON EXPRESS,AG3,2022-09-30 00:00:00,2022-09-30 10:20:00,2022-10-01 22:00:00,2303,1866,26,DEPARTED,SPXX
166,T1(P),MSC,MEXP001,XA239A/HW238A,02 (15) 16,MSC EXPRESS III,PERTWI,2022-10-04 20:00:00,2022-10-05 10:00:00,2022-10-06 03:00:00,0,1075,0,DEPARTED,MEXP


In [191]:
# df_pp.loc[df_pp[df_pp["vessel_name"] == 'RIODEJANEIRO EXPRESS'].index,"vessel_name"] ='RIO DE JANEIRO EXPRESS'

In [223]:
# df_pp.loc[df_pp[df_pp["vessel_name"] == 'ZHUCHENGXINZHOU'].index,"vessel_name"] ='ZHU CHENG XIN ZHOU'

In [247]:
# df_pp.loc[df_pp[df_pp["vessel_name"] == 'ZHEUNG-A ULSAN'].index,"vessel_name"] ='HEUNG-A ULSAN'

In [229]:
# df_pp.loc[df_pp[df_pp["vessel_name"] == 'MSC MUNDRA VIII'].index,"vessel_name"] ='MSC MUNDRA'

In [235]:
# df_pp.loc[df_pp[df_pp["vessel_name"] == 'LONG BEACH TRADER'].index,"vessel_name"] ='MSC LONG BEACH VI'

In [241]:
pnit_berth_history_df.query("선명.str.contains('DANIT')")

,선석,선사,모선항차,선사항차,Head (Bridge) Stern,선명,ROUTE,반입마감시한,접안(예정)일시,출항(예정)일시,양하,적하,Shift,상태,VSL_CD


In [201]:
pnit_berth_history_df[pnit_berth_history_df["선사"] == "MSC"].tail(50)
# pnit_berth_history_df[pnit_berth_history_df["선사"] == "MSC"].query("선명.str.contains('TORONTO')")


,선석,선사,모선항차,선사항차,Head (Bridge) Stern,선명,ROUTE,반입마감시한,접안(예정)일시,출항(예정)일시,양하,적하,Shift,상태,VSL_CD
187,T1(P),MSC,MVBN001,XA235A/XA235A,01 (16) 21,MSC BUSAN,SPT,2022-10-16 18:00:00,2022-10-17 04:20:00,2022-10-18 01:00:00,2784,0,0,DEPARTED,MVBN
188,T2(P),MSC,MWSH003,FY241A/FY241A,21 (29) 42,MSC WASHINGTON,AFRICA,2022-10-16 17:00:00,2022-10-17 04:45:00,2022-10-17 20:00:00,759,561,0,DEPARTED,MWSH
189,T3(S),MSC,MLLS004,UK240A/UK240A,63 (50) 45,MSC LISA,CHINK,2022-10-18 06:00:00,2022-10-17 07:20:00,2022-10-22 22:00:00,156,412,384,DEPARTED,MLLS
190,T2(P),MSC,MIDA018,HW239R/HW241A,23 (32) 36,MSC IDA II,PERTWI,2022-10-17 14:00:00,2022-10-17 22:00:00,2022-10-18 16:00:00,273,689,40,DEPARTED,MIDA
192,T2(P),MSC,MPGS002,UX239A/UX239A,25 (38) 43,MSC PEGASUS,SANTA,2022-10-18 08:00:00,2022-10-18 18:30:00,2022-10-20 00:00:00,27,2999,0,DEPARTED,MPGS
195,T1(P),MSC,MSJU002,UK241A/UK241A,01 (15) 21,MSC JUDITH,CHINK,2022-10-20 15:00:00,2022-10-21 01:30:00,2022-10-22 02:00:00,300,963,4,DEPARTED,MSJU
196,T2(P),MSC,MRAP002,FA235R/FA235R,24 (33) 46,MSC RAPALLO,ASPA2W,2022-10-20 22:00:00,2022-10-21 08:00:00,2022-10-22 12:00:00,1634,187,208,DEPARTED,MRAP
199,T3(P),MSC,MRII001,HW240R/HW242A,49 (59) 63,MSC RINI III,PERTWI,2022-10-22 21:00:00,2022-10-23 07:10:00,2022-10-24 01:00:00,234,1086,0,DEPARTED,MRII
202,T3(P),MSC,MLPZ003,FY242A/FY242A,41 (50) 63,MSC LA SPEZIA,AFRICA,2022-10-23 19:00:00,2022-10-24 03:02:00,2022-10-25 02:00:00,1399,694,50,DEPARTED,MLPZ
203,T2(P),MSC,MTOA007,FZ232R/FZ232R,23 (37) 42,MSC ANTONELLA,AN3W,2022-10-24 21:00:00,2022-10-25 07:30:00,2022-10-26 11:00:00,1429,426,124,DEPARTED,MTOA


In [173]:
no_port_mis_name_2

['RIODEJANEIRO EXPRESS',
 'MSC CHANNE',
 'MSC BRANKA',
 'MSC CHARLESTON',
 'MSC TORONTO',
 'COSCO NEW YORK',
 'MSC SOLA',
 'NAVIOS JASMINE',
 'MSC SHAY',
 'ZHUCHENGXINZHOU',
 'HEUNGA ULSAN',
 'RIO GRANDE EXPRESS',
 'COSCO SAO PAULO',
 'PARANAGUA EXPRESS',
 'MSC POLARIS',
 'MSC TANIA',
 'MSC ANTONIA',
 'MSC ELLEN',
 'MSC MUNDRA VIII',
 'MSC EVEREST',
 'MSC NIMISHA III',
 'MSC BETTINA',
 'MSC FREEPORT',
 'MSC MOON F',
 'MSC DEILA',
 'MSC MONICA',
 'MSC ALICE',
 'MSC MICHIGAN VII',
 'MSC MONTEREY',
 'MSC HEIDI',
 'MSC TRADER II',
 'MSC TIPHAINE',
 'MSC MEXICO V',
 'MSC BARBADOS',
 'MSC ELSA3',
 'LONG BEACH TRADER',
 'MSC VILDA X',
 'MSC PETRA',
 'MSC REBECCA',
 'MSC FELIXSTOWE',
 'MSC DARWIN VI',
 'MSC ORNELLA',
 'MSC NERISSA',
 'MSC TRIESTE',
 'MSC ROBERTA   V',
 'MSC INDIA',
 'XIN NAN SHA',
 'MSC ANS',
 'MSC DANIT']

In [205]:
DT_VSSL_ARVL_F.query(f'CALL_LETTER == "VRTZ3"')

,VSSL_KEY,YR,SER_NO,RPT_DT,PERF_DT,BERTH_PLC_CODE_1,BERTH_PLC_CODE_2,NXT_PRT_1,NXT_PRT_2,TRANS_CRG_TN,DNGR_CRG_TN,GRS_TN,INT_TN,FLAG,VSSL_TP,VSSL_NM,SUB_CALL_LETTER_1,SUB_CALL_LETTER_2,SUB_TN_1,SUB_TN_2,ARVL_OBJ_CODE,ETD_DT,LAST_PRT_DEP_DT,LAST_PRT_1,LAST_PRT_2,ARVL_DT,CALL_LETTER,DEAD_WEIGHT,CRG_TN,UPDT_DATE


In [212]:
port_mis_raw_df[port_mis_raw_df["호출부호"] == 'BTSJ']

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
306,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,22,최종,외항,출항,"25,703",2022-07-22 23:40,2022-07-23 21:54,2022-07-23 22:00,2022-07-25 09:13,기타,22MSCU0648E,MSN,06,신항 2부두 3선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,21,NaN,Y,Y,NaN,NaN
307,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,22,최종,외항,입항,"25,703",2022-07-22 23:40,2022-07-23 21:54,2022-07-23 00:00,2022-07-25 09:06,기타,22MSCU1121I,MSN,06,신항 2부두 3선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,21,NaN,Y,Y,NaN,NaN
768,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,23,최종,외항,출항,"25,703",2022-07-29 11:55,2022-07-30 12:50,2022-07-30 13:00,2022-08-01 08:29,기타,22MSCU0660E,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,21,NaN,Y,Y,NaN,NaN
769,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,23,최종,외항,입항,"25,703",2022-07-29 11:55,2022-07-30 12:50,2022-07-29 12:00,2022-07-29 12:49,기타,22MSCU1122I,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,21,NaN,Y,Y,NaN,NaN
1222,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,24,최종,외항,출항,"25,703",2022-08-06 00:02,2022-08-06 22:00,2022-08-06 22:00,2022-08-08 09:05,기타,22MSCU0908E,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,21,NaN,Y,Y,NaN,NaN
1223,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,24,최종,외항,입항,"25,703",2022-08-06 00:02,2022-08-06 22:00,2022-08-06 00:00,2022-08-08 08:51,기타,22MSCU1151I,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,21,NaN,Y,Y,NaN,NaN
1648,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,25,최종,외항,입항,"25,703",2022-08-12 07:55,2022-08-13 07:00,2022-08-12 08:00,2022-08-12 09:04,기타,22MSCU1152I,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,20,NaN,Y,Y,NaN,NaN
1649,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,25,최종,외항,출항,"25,703",2022-08-12 07:55,2022-08-13 07:00,2022-08-13 07:00,2022-08-16 08:57,기타,22MSCU0930E,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,20,NaN,Y,Y,NaN,NaN
2032,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,26,최종,외항,입항,"25,703",2022-08-18 01:00,2022-08-18 23:00,2022-08-18 01:00,2022-08-19 11:12,기타,22MSCU1153I,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,20,NaN,Y,Y,NaN,NaN
2033,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,26,최종,외항,출항,"25,703",2022-08-18 01:00,2022-08-18 23:00,2022-08-18 23:00,2022-08-19 11:12,기타,22MSCU0933E,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA,풀컨테이너선,NaN,20,NaN,Y,Y,NaN,NaN


In [209]:
for name in pnit_berth_history_df["선명"].values:
    d = DT_VSSL_ARVL_F.query(f"VSSL_NM == '{name}'")
    if len(d) == 0:
        print(name)

ZHU CHENG XIN ZHOU
ZHU CHENG XIN ZHOU
MSC MEXICO V


In [ ]:
MSC MEXICO V


In [211]:
pnit_berth_history_df[pnit_berth_history_df["선명"] == 'ZHU CHENG XIN ZHOU']

,선석,선사,모선항차,선사항차,Head (Bridge) Stern,선명,ROUTE,반입마감시한,접안(예정)일시,출항(예정)일시,양하,적하,Shift,상태,VSL_CD
256,T2(P),MSC,MZXZ039,HG245A/HG245R,24 (36) 38,ZHU CHENG XIN ZHOU,KAGUYA,2022-11-18 07:00:00,2022-11-18 20:30:00,2022-11-19 13:00:00,224,747,0,DEPARTED,MZXZ
273,T1(P),MSC,MZXZ040,HG246A/HG246R,05 (17) 19,ZHU CHENG XIN ZHOU,KAGUYA,2022-11-26 07:00:00,2022-11-26 17:00:00,2022-11-27 09:00:00,365,912,0,PLANNED,MZXZ


In [303]:
df_pp["vessel_name"].value_counts()

0.000000       275
12.018995       15
11.996725       13
12.019664       12
12.018333       12
              ... 
842.077667       1
7390.685043      1
3857.091418      1
1968.082180      1
5892.948969      1
Name: vessel_name, Length: 125500, dtype: int64

너무 데이터가 없는데..
1. pnit 접안 이력이 있는 선박 이름 기준으로 구하기
2. (띄워쓰기 같은거 때문 말고) 기간 내 port-mis 데이터가 있는 (= 부산 접안 이력) 선박들 이름 기준으로 구하기

## pnit 접안 이력 데이터

### pnit에 접안 이력은 있으나 AIS 에 없는 것들

In [248]:
for name in list(dict.fromkeys(pnit_berth_history_df["선명"].values)):
    if len(df_pp.query(f"vessel_name == '{name}'")) == 0:
        print(name)

SEAMAX FAIRFIELD
MSC DESIREE
MSC BREMEN
NAVIOS UTMOST
MSC MARGARITA
LEO C
MSC TAMPA
OOCL BRAZIL
MSC TIANPING
YM WELCOME
XIN MEI ZHOU
MSC NASSAU
CONTI ANNAPURNA
MSC AURIGA
CONTI CHIVALRY
MSC VITA
MSC LOS ANGELES
SONGA TIGER


이유1 : 데이터가 너무 적게 들어와서 걸러짐   
이유2 : AIS 데이터 업데이트가 오랫동안 되지 않아 걸러짐   
이유3 : 선박의 이름과 AIS 데이터의 선박의 이름이 다름   
이유4 : AIS 데이터에 결측치가 있어서 제거됨.    
이유5 : AIS 시간 차이가 너무 많이 나서 제거됨.

### pnit 접안 이력 + AIS도 존재

In [270]:
new_whole_list = []
for name in list(dict.fromkeys(pnit_berth_history_df["선명"].values)):
    if len(df_pp.query(f"vessel_name == '{name}'")) != 0:
        new_whole_list.append(name)

In [294]:
len(new_whole_list)

137

137척의 선박 체크

In [305]:
# df_pp.columns = [
#     'remained_distance', 'speed', 'time_voyage_data_received', 'destination',
#     'estimated_fuel_consumption', 'vessel_name', 'mmsi', 'navigation_info',
#     'timestamp', 'latitude', 'time_position_data_received', 'imo', 'heading',
#     'eta', 'CAPACITY', 'longitude'
# ]

In [308]:
df_pp = df_pp[['vessel_name', 'mmsi', 'imo', 'CAPACITY', 'destination', 'eta',
    'longitude', 'latitude', 'speed', 'heading', 'timestamp',
    'time_position_data_received', 'time_voyage_data_received',
    'navigation_info', 'remained_distance', 'estimated_fuel_consumption'
]]
    

In [322]:
# df_pp.to_csv("항적데이터/filtered_ais.csv", encoding="utf-8 sig", header=True, index=False)

In [312]:
for name in new_whole_list:
    name2 = name.replace(" ","_")
    globals()[f"{name2}_voyage_df"]  = df_pp.query(f"vessel_name == '{name}'")


In [313]:
# 저장할 공간
path = "항적데이터"

In [314]:
for i in globals():
    if "_voyage_df" in i:
        f = globals()[i]
        f.reset_index(inplace=True, drop=True)
        f.to_csv(path+"/"+str(i)+".csv", encoding="utf-8 sig", header=True, index=False)

In [317]:
path_list = os.listdir(path)
path_list

['AKADIMOS_voyage_df.csv',
 'AKA_BHUM_voyage_df.csv',
 'AMALTHEA_voyage_df.csv',
 'CAPE_ARTEMISIO_voyage_df.csv',
 'CAUQUENES_voyage_df.csv',
 'CAUTIN_voyage_df.csv',
 'CISNES_voyage_df.csv',
 'COCHRANE_voyage_df.csv',
 'COPIAPO_voyage_df.csv',
 'COSCO_ASIA_voyage_df.csv',
 'COSCO_FOS_voyage_df.csv',
 'COSCO_PRINCE_RUPERT_voyage_df.csv',
 'COYHAIQUE_voyage_df.csv',
 'CSCL_ASIA_voyage_df.csv',
 'CSCL_AUTUMN_voyage_df.csv',
 'CSCL_WINTER_voyage_df.csv',
 'HENG_HUI_6_voyage_df.csv',
 'HEUNG-A_ULSAN_voyage_df.csv',
 'HMM_BLESSING_voyage_df.csv',
 'HMM_RAON_voyage_df.csv',
 'HYUNDAI_SATURN_voyage_df.csv',
 'KUALA_LUMPUR_EXPRESS_voyage_df.csv',
 'MANZANILLO_EXPRESS_voyage_df.csv',
 'MERSIN_EXPRESS_voyage_df.csv',
 'MSC_ABY_voyage_df.csv',
 'MSC_AGRIGENTO_voyage_df.csv',
 'MSC_ALGHERO_voyage_df.csv',
 'MSC_ALINA_voyage_df.csv',
 'MSC_ALIYA_voyage_df.csv',
 'MSC_AMEERA_III_voyage_df.csv',
 'MSC_ANAHITA_voyage_df.csv',
 'MSC_ANA_CAMILA_III_voyage_df.csv',
 'MSC_ANDREA_F_voyage_df.csv',
 'MSC_AN

In [321]:
df_pp["vessel_name"].value_counts()

ZHU CHENG XIN ZHOU    4056
COSCO FOS             3803
HEUNG-A ULSAN         3730
PACIFIC SINGAPORE     3349
SINOKOR NIIGATA       3311
                      ... 
MSC BETTINA             66
MSC CAPELLA             64
UMM SALAL               61
YM WELLBEING            52
MSC BEIJING             51
Name: vessel_name, Length: 171, dtype: int64